# Part of the code for R2M model

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sat Nov 13 21:30:28 2021

@author: xiaweijie
"""
import pandas as pd
import numpy as np
import pickle
import csv


#read data
df_generation = pd.read_csv(r'Energy flux.csv')
data = np.array(df_generation.loc[:,['Hours per Day','Hours Per Year','PV Generation','CES CONSUMPTION']])
surplus = []
grid_con = []
for i in range(len(data)):
    a=data[i,2]-data[i,3]
    if a <=0:
        surplus.append(0)
        grid_con.append(-a)
    else:
        surplus.append(a)
        grid_con.append(0)

surplus = np.array(surplus)
grid_con = np.array(grid_con)
s_g=np.transpose(np.vstack((surplus,grid_con)))
data_all_or = np.hstack((data,s_g))
data_all_or1 = np.copy(data_all_or)
data_all_or2 = np.copy(data_all_or)

# Split PV generation 
pv_consumer1=1 # percentage for consumer1
pv_consumer2=0 # percentage for consumer2
for i in range(len(data_all_or)):
    data_all_or1[i,2]= pv_consumer1*data_all_or[i,2]
    data_all_or2[i,2]= pv_consumer2*data_all_or[i,2]
    if data_all_or1[i,2]-data_all_or1[i,3]<0:
        data_all_or1[i,4]=0
    else:
        data_all_or1[i,4]=data_all_or1[i,2]-data_all_or1[i,3]
    if data_all_or2[i,2]-data_all_or2[i,3]<0:
        data_all_or2[i,4]=0
    else:
        data_all_or2[i,4]=data_all_or2[i,2]-data_all_or2[i,3]


"Consumer 1"
#battery static parameters
capacity1 = 10000  #Wh batter's capacity1
deep1=0.05  #batter discharge at most to capacity1*deep1 of its capacity1
dis_deep1= capacity1*deep1 
bat_charge_speed1 = 1000 #w charging speed
char_time_start1 = 4  #charging start time
char_time_stop = 7   #charging stop time


#battery dynamic parameters
cap_dy1 = 0  #capacity1 of battery dynamic
en_dump1 = [] #dumped energy
en_battery_grid1 = [] #battery's energy from grid
en_battery_pv1 = [] #battery's energy from PV
battery_stat1 = []
c=0

#power dynamic parameters
use_power_pv1 = [] # the energy generated by pV and be used not dumped
use_power_grid1 = [] # the energy comes from grid


for x in range(len(data)):
    if char_time_stop >= data_all_or1[x,0] >= char_time_start1: #battery at its charging time
        if data_all_or1[x,4] > 0: #surplus >0
            if cap_dy1 < capacity1: #battery is not full
                if data_all_or1[x,4] > bat_charge_speed1: #surplus > charging speed
                    if capacity1-cap_dy1 < bat_charge_speed1: #full-capacity1 now<speed
                        # battery parameters
                        en_dump1.append(data_all_or1[x,4] - (capacity1 - cap_dy1))
                        en_battery_grid1.append(0)
                        en_battery_pv1.append(capacity1 - cap_dy1)                  
                        # power paremeters
                        use_power_pv1.append(data_all_or1[x,2]-(data_all_or1[x,4] - (capacity1 - cap_dy1)))
                        use_power_grid1.append(0)
                        # battery cpacity
                        cap_dy1 = capacity1
                        battery_stat1.append(cap_dy1)
                    else: #full-capacity1 now > speed
                        # battery parameters
                        en_dump1.append(data_all_or1[x,4]-bat_charge_speed1)
                        en_battery_grid1.append(0)
                        en_battery_pv1.append(bat_charge_speed1)
                        # power paremeters
                        use_power_pv1.append(data_all_or1[x,2]-(data_all_or1[x,4]-bat_charge_speed1))
                        use_power_grid1.append(0)
                        # battery cpacity
                        cap_dy1 += bat_charge_speed1
                        battery_stat1.append(cap_dy1)
                else: #surplus <= charging speed
                    if capacity1-cap_dy1 < data_all_or1[x,4]: #full-capacity1 now<surplus
                        # battery parameters
                        en_battery_grid1.append(0)
                        en_dump1.append(data_all_or1[x,4] - (capacity1 - cap_dy1))
                        en_battery_pv1.append(capacity1 - cap_dy1)
                        # power paremeters
                        use_power_pv1.append(data_all_or1[x,2] - (data_all_or1[x,4] - (capacity1 - cap_dy1)) )
                        use_power_grid1.append(0)
                        # battery cpacity
                        cap_dy1 = capacity1
                        battery_stat1.append(cap_dy1)
                    else: #full-capacity1 now > surplus
                        if capacity1-cap_dy1 < bat_charge_speed1:  #full-capacity1 now<speed
                            # battery parameters
                            en_dump1.append(0)
                            en_battery_grid1.append(capacity1-cap_dy1-data_all_or1[x,4])
                            en_battery_pv1.append(data_all_or1[x,4])
                            # power paremeters
                            use_power_pv1.append(data_all_or1[x,2])
                            use_power_grid1.append(capacity1-cap_dy1-data_all_or1[x,4])
                            # battery cpacity
                            cap_dy1 = capacity1
                            battery_stat1.append(cap_dy1)
                        else: #full-capacity1 now>speed
                            # battery parameters
                            en_dump1.append(0)
                            en_battery_grid1.append(bat_charge_speed1-data_all_or1[x,4])
                            en_battery_pv1.append(data_all_or1[x,4])
                            # power paremeters
                            use_power_pv1.append(data_all_or1[x,2])
                            use_power_grid1.append(bat_charge_speed1-data_all_or1[x,4])
                            # battery cpacity
                            cap_dy1 += bat_charge_speed1
                            battery_stat1.append(cap_dy1)                     
            else: #battery is full
                 # battery parameters
                 en_dump1.append(data_all_or1[x,4])
                 en_battery_grid1.append(0)
                 en_battery_pv1.append(0)
                 # power paremeters
                 use_power_pv1.append(data_all_or1[x,2]-data_all_or1[x,4])
                 use_power_grid1.append(0)
                 # battery cpacity
                 cap_dy1 = cap_dy1
                 battery_stat1.append(cap_dy1)  
        else: #surplus = 0
            if cap_dy1 < capacity1: #battery is not full
                if capacity1-cap_dy1 < bat_charge_speed1: #full-capacity1 now<speed
                    # battery parameters
                    en_dump1.append(0)
                    en_battery_grid1.append(capacity1-cap_dy1)
                    en_battery_pv1.append(0)
                    # power paremeters
                    use_power_pv1.append(data_all_or1[x,2])
                    use_power_grid1.append(data_all_or1[x,3]-data_all_or1[x,2]+capacity1-cap_dy1)
                    # battery cpacity
                    cap_dy1=cap_dy1
                    battery_stat1.append(cap_dy1)  
                else: #full-capacity1 now>=speed
                    # battery parameters
                    en_dump1.append(0)
                    en_battery_grid1.append(bat_charge_speed1)
                    en_battery_pv1.append(0)
                    # power paremeters
                    use_power_pv1.append(data_all_or1[x,2])
                    use_power_grid1.append(data_all_or1[x,3]-data_all_or1[x,2]+bat_charge_speed1)
                    # battery cpacity
                    cap_dy1 += bat_charge_speed1
                    battery_stat1.append(cap_dy1)                 
            else: #battery is full
                    # battery parameters
                    en_dump1.append(0)   
                    en_battery_grid1.append(0)
                    en_battery_pv1.append(0)
                    # power paremeters
                    use_power_pv1.append(data_all_or1[x,2])
                    use_power_grid1.append(data_all_or1[x,3]-data_all_or1[x,2]+0)
                    # battery cpacity
                    cap_dy1 += 0
                    battery_stat1.append(cap_dy1)                 
    else:
        if data_all_or1[x,4] >0: #surplus>0
            if cap_dy1 < capacity1: #battery not full
                if data_all_or1[x,4] > bat_charge_speed1:  #surplus>charge speed 
                    if bat_charge_speed1 > capacity1 - cap_dy1:  #charge speed >full-cap_dy1
                        # battery parameters
                        en_dump1.append(data_all_or1[x,4]-(capacity1 - cap_dy1))   
                        en_battery_grid1.append(0)
                        en_battery_pv1.append(capacity1 - cap_dy1)
                        # power paremeters
                        use_power_pv1.append(data_all_or1[x,2]-(data_all_or1[x,4]-(capacity1 - cap_dy1)))
                        use_power_grid1.append(0)
                        # battery cpacity
                        cap_dy1 = capacity1
                        battery_stat1.append(cap_dy1)    
                    else: #charge speed < full-cap_dy1
                        # battery parameters
                        en_dump1.append(data_all_or1[x,4]-bat_charge_speed1)   
                        en_battery_grid1.append(0)
                        en_battery_pv1.append(bat_charge_speed1)
                        # power paremeters
                        use_power_pv1.append(data_all_or1[x,2]-(data_all_or1[x,4]-bat_charge_speed1))
                        use_power_grid1.append(0)
                        # battery cpacity
                        cap_dy1 += bat_charge_speed1
                        battery_stat1.append(cap_dy1)  
                else: #surplus < charge speed 
                    if data_all_or1[x,4] > capacity1 - cap_dy1: #surplus > full-cap_dy1
                        # battery parameters
                        en_dump1.append(data_all_or1[x,4]-(capacity1 - cap_dy1))   
                        en_battery_grid1.append(0)
                        en_battery_pv1.append(capacity1 - cap_dy1)
                        # power paremeters
                        use_power_pv1.append(data_all_or1[x,2]-(data_all_or1[x,4]-(capacity1 - cap_dy1)))
                        use_power_grid1.append(0)
                        # battery cpacity
                        cap_dy1 = capacity1
                        battery_stat1.append(cap_dy1)
                    else: #surplus < full-cap_dy1
                        # battery parameters
                        en_dump1.append(0)   
                        en_battery_grid1.append(0)
                        en_battery_pv1.append(data_all_or1[x,4])
                        # power paremeters
                        use_power_pv1.append(data_all_or1[x,2])
                        use_power_grid1.append(0)
                        # battery cpacity
                        cap_dy1 += data_all_or1[x,4]
                        battery_stat1.append(cap_dy1)
            else: #battery is full
                # battery parameters
                en_dump1.append(data_all_or1[x,4])   
                en_battery_grid1.append(0)
                en_battery_pv1.append(0)
                # power paremeters
                use_power_pv1.append(data_all_or1[x,2]-data_all_or1[x,4])
                use_power_grid1.append(0)
                # battery cpacity
                cap_dy1 =  cap_dy1
                battery_stat1.append(cap_dy1)
        else: #surplus=0
            if cap_dy1-dis_deep1 > 0: #battery can work
                if cap_dy1-dis_deep1+data_all_or1[x,2] > data_all_or1[x,3] : #no grid comsunption
                    # battery parameters
                    en_dump1.append(0)   
                    en_battery_grid1.append(0)
                    en_battery_pv1.append(0)
                    # power paremeters
                    use_power_pv1.append(data_all_or1[x,2])
                    use_power_grid1.append(0)
                    # battery cpacity
                    cap_dy1 =  cap_dy1-data_all_or1[x,3]-dis_deep1+data_all_or1[x,2]
                    battery_stat1.append(cap_dy1)
                else: #there is grid comsunption
                    # battery parameters
                    en_dump1.append(0)   
                    en_battery_grid1.append(0)
                    en_battery_pv1.append(0)
                    # power paremeters
                    use_power_pv1.append(data_all_or1[x,2])
                    use_power_grid1.append(data_all_or1[x,3]-data_all_or1[x,2]-(cap_dy1-dis_deep1))
                    # battery cpacity
                    cap_dy1 =  dis_deep1
                    battery_stat1.append(cap_dy1)
            else: #battery can not work
                 # battery parameters
                en_dump1.append(0)   
                en_battery_grid1.append(0)
                en_battery_pv1.append(0)
                # power paremeters
                use_power_pv1.append(data_all_or1[x,2])
                use_power_grid1.append(data_all_or1[x,3]-data_all_or1[x,2])
                # battery cpacity
                cap_dy1 =  cap_dy1
                battery_stat1.append(cap_dy1)
        
#the table of consumer1
#data_all_or1(time,hour,generation,consumption,surplus,grid consumption)
#use_power_pv1,use_power_grdi1
#battery_stat1,en_battery_grid1, en_battery_pv1, en_dump1,

#change the format of data
use_power_pv1=np.array(use_power_pv1)
use_power_grid1=np.array(use_power_grid1)
battery_stat1=np.array(battery_stat1)
en_battery_grid1=np.array(en_battery_grid1)
en_batteryp_v1=np.array(en_battery_pv1)
en_dump1=np.array(en_dump1)

head1=np.array(['time_day','hour','generation_pv','consumption','surplus','grid consumption','use_power_pv1','use_power_grid1',
               'battery_stat1','en_battery_grid1', 'en_battery_pv1', 'en_dump1'])
data1_power_battery=np.transpose(np.vstack((use_power_pv1,use_power_grid1,battery_stat1,en_battery_grid1,en_batteryp_v1,en_dump1))) 
data1=np.vstack((head1,np.hstack((data_all_or1,data1_power_battery))))


            
"Consumer 2"

#battery static parameters
capacity2 = 10000  #Wh batter's capacity2
deep2=0.05  #batter discharge at most to capacity2*deep2 of its capacity2
dis_deep2 = capacity2*deep2 
bat_charge_speed2 = 1000 #w charging speed
char_time_start2 = 4  #charging start time
char_time_stop2 = 7   #charging stop time


#battery dynamic parameters
cap_dy2 = 0  #capacity2 of battery dynamic
en_dump2 = [] #dumped energy
en_battery_grid2 = [] #battery's energy from grid
en_battery_pv2 = [] #battery's energy from PV
battery_stat2 = []
c=0

#power dynamic parameters
use_power_pv2 = [] # the energy generated by pV and be used not dumped
use_power_grid2 = [] # the energy comes from grid


for x in range(len(data)):
    if char_time_stop2 >= data_all_or2 [x,0] >= char_time_start2: #battery at its charging time
        if data_all_or2 [x,4] > 0: #surplus >0
            if cap_dy2 < capacity2: #battery is not full
                if data_all_or2 [x,4] > bat_charge_speed2: #surplus > charging speed
                    if capacity2-cap_dy2 < bat_charge_speed2: #full-capacity2 now<speed
                        # battery parameters
                        en_dump2.append(data_all_or2 [x,4] - (capacity2 - cap_dy2))
                        en_battery_grid2.append(0)
                        en_battery_pv2.append(capacity2 - cap_dy2)                  
                        # power paremeters
                        use_power_pv2.append(data_all_or2 [x,2]-(data_all_or2 [x,4] - (capacity2 - cap_dy2)))
                        use_power_grid2.append(0)
                        # battery cpacity
                        cap_dy2 = capacity2
                        battery_stat2.append(cap_dy2)
                    else: #full-capacity2 now > speed
                        # battery parameters
                        en_dump2.append(data_all_or2 [x,4]-bat_charge_speed2)
                        en_battery_grid2.append(0)
                        en_battery_pv2.append(bat_charge_speed2)
                        # power paremeters
                        use_power_pv2.append(data_all_or2 [x,2]-(data_all_or2 [x,4]-bat_charge_speed2))
                        use_power_grid2.append(0)
                        # battery cpacity
                        cap_dy2 += bat_charge_speed2
                        battery_stat2.append(cap_dy2)
                else: #surplus <= charging speed
                    if capacity2-cap_dy2 < data_all_or2 [x,4]: #full-capacity2 now<surplus
                        # battery parameters
                        en_battery_grid2.append(0)
                        en_dump2.append(data_all_or2 [x,4] - (capacity2 - cap_dy2))
                        en_battery_pv2.append(capacity2 - cap_dy2)
                        # power paremeters
                        use_power_pv2.append(data_all_or2 [x,2] - (data_all_or2 [x,4] - (capacity2 - cap_dy2)) )
                        use_power_grid2.append(0)
                        # battery cpacity
                        cap_dy2 = capacity2
                        battery_stat2.append(cap_dy2)
                    else: #full-capacity2 now > surplus
                        if capacity2-cap_dy2 < bat_charge_speed2:  #full-capacity2 now<speed
                            # battery parameters
                            en_dump2.append(0)
                            en_battery_grid2.append(capacity2-cap_dy2-data_all_or2 [x,4])
                            en_battery_pv2.append(data_all_or2 [x,4])
                            # power paremeters
                            use_power_pv2.append(data_all_or2 [x,2])
                            use_power_grid2.append(capacity2-cap_dy2-data_all_or2 [x,4])
                            # battery cpacity
                            cap_dy2 = capacity2
                            battery_stat2.append(cap_dy2)
                        else: #full-capacity2 now>speed
                            # battery parameters
                            en_dump2.append(0)
                            en_battery_grid2.append(bat_charge_speed2-data_all_or2 [x,4])
                            en_battery_pv2.append(data_all_or2 [x,4])
                            # power paremeters
                            use_power_pv2.append(data_all_or2 [x,2])
                            use_power_grid2.append(bat_charge_speed2-data_all_or2 [x,4])
                            # battery cpacity
                            cap_dy2 += bat_charge_speed2
                            battery_stat2.append(cap_dy2)                     
            else: #battery is full
                 # battery parameters
                 en_dump2.append(data_all_or2 [x,4])
                 en_battery_grid2.append(0)
                 en_battery_pv2.append(0)
                 # power paremeters
                 use_power_pv2.append(data_all_or2 [x,2]-data_all_or2 [x,4])
                 use_power_grid2.append(0)
                 # battery cpacity
                 cap_dy2 = cap_dy2
                 battery_stat2.append(cap_dy2)  
        else: #surplus = 0
            if cap_dy2 < capacity2: #battery is not full
                if capacity2-cap_dy2 < bat_charge_speed2: #full-capacity2 now<speed
                    # battery parameters
                    en_dump2.append(0)
                    en_battery_grid2.append(capacity2-cap_dy2)
                    en_battery_pv2.append(0)
                    # power paremeters
                    use_power_pv2.append(data_all_or2 [x,2])
                    use_power_grid2.append(data_all_or2 [x,3]-data_all_or2 [x,2]+capacity2-cap_dy2)
                    # battery cpacity
                    cap_dy2=cap_dy2
                    battery_stat2.append(cap_dy2)  
                else: #full-capacity2 now>=speed
                    # battery parameters
                    en_dump2.append(0)
                    en_battery_grid2.append(bat_charge_speed2)
                    en_battery_pv2.append(0)
                    # power paremeters
                    use_power_pv2.append(data_all_or2 [x,2])
                    use_power_grid2.append(data_all_or2 [x,3]-data_all_or2 [x,2]+bat_charge_speed2)
                    # battery cpacity
                    cap_dy2 += bat_charge_speed2
                    battery_stat2.append(cap_dy2)                 
            else: #battery is full
                    # battery parameters
                    en_dump2.append(0)   
                    en_battery_grid2.append(0)
                    en_battery_pv2.append(0)
                    # power paremeters
                    use_power_pv2.append(data_all_or2 [x,2])
                    use_power_grid2.append(data_all_or2 [x,3]-data_all_or2 [x,2]+0)
                    # battery cpacity
                    cap_dy2 += 0
                    battery_stat2.append(cap_dy2)                 
    else:
        if data_all_or2 [x,4] >0: #surplus>0
            if cap_dy2 < capacity2: #battery not full
                if data_all_or2 [x,4] > bat_charge_speed2:  #surplus>charge speed 
                    if bat_charge_speed2 > capacity2 - cap_dy2:  #charge speed >full-cap_dy2
                        # battery parameters
                        en_dump2.append(data_all_or2 [x,4]-(capacity2 - cap_dy2))   
                        en_battery_grid2.append(0)
                        en_battery_pv2.append(capacity2 - cap_dy2)
                        # power paremeters
                        use_power_pv2.append(data_all_or2 [x,2]-(data_all_or2 [x,4]-(capacity2 - cap_dy2)))
                        use_power_grid2.append(0)
                        # battery cpacity
                        cap_dy2 = capacity2
                        battery_stat2.append(cap_dy2)    
                    else: #charge speed < full-cap_dy2
                        # battery parameters
                        en_dump2.append(data_all_or2 [x,4]-bat_charge_speed2)   
                        en_battery_grid2.append(0)
                        en_battery_pv2.append(bat_charge_speed2)
                        # power paremeters
                        use_power_pv2.append(data_all_or2 [x,2]-(data_all_or2 [x,4]-bat_charge_speed2))
                        use_power_grid2.append(0)
                        # battery cpacity
                        cap_dy2 += bat_charge_speed2
                        battery_stat2.append(cap_dy2)  
                else: #surplus < charge speed 
                    if data_all_or2 [x,4] > capacity2 - cap_dy2: #surplus > full-cap_dy2
                        # battery parameters
                        en_dump2.append(data_all_or2 [x,4]-(capacity2 - cap_dy2))   
                        en_battery_grid2.append(0)
                        en_battery_pv2.append(capacity2 - cap_dy2)
                        # power paremeters
                        use_power_pv2.append(data_all_or2 [x,2]-(data_all_or2 [x,4]-(capacity2 - cap_dy2)))
                        use_power_grid2.append(0)
                        # battery cpacity
                        cap_dy2 = capacity2
                        battery_stat2.append(cap_dy2)
                    else: #surplus < full-cap_dy2
                        # battery parameters
                        en_dump2.append(0)   
                        en_battery_grid2.append(0)
                        en_battery_pv2.append(data_all_or2 [x,4])
                        # power paremeters
                        use_power_pv2.append(data_all_or2 [x,2])
                        use_power_grid2.append(0)
                        # battery cpacity
                        cap_dy2 += data_all_or2 [x,4]
                        battery_stat2.append(cap_dy2)
            else: #battery is full
                # battery parameters
                en_dump2.append(data_all_or2 [x,4])   
                en_battery_grid2.append(0)
                en_battery_pv2.append(0)
                # power paremeters
                use_power_pv2.append(data_all_or2 [x,2]-data_all_or2 [x,4])
                use_power_grid2.append(0)
                # battery cpacity
                cap_dy2 =  cap_dy2
                battery_stat2.append(cap_dy2)
        else: #surplus=0
            if cap_dy2-dis_deep2 > 0: #battery can work
                if cap_dy2-dis_deep2+data_all_or2 [x,2] > data_all_or2 [x,3] : #no grid comsunption
                    # battery parameters
                    en_dump2.append(0)   
                    en_battery_grid2.append(0)
                    en_battery_pv2.append(0)
                    # power paremeters
                    use_power_pv2.append(data_all_or2 [x,2])
                    use_power_grid2.append(0)
                    # battery cpacity
                    cap_dy2 =  cap_dy2-data_all_or2 [x,3]-dis_deep2+data_all_or2 [x,2]
                    battery_stat2.append(cap_dy2)
                else: #there is grid comsunption
                    # battery parameters
                    en_dump2.append(0)   
                    en_battery_grid2.append(0)
                    en_battery_pv2.append(0)
                    # power paremeters
                    use_power_pv2.append(data_all_or2 [x,2])
                    use_power_grid2.append(data_all_or2 [x,3]-data_all_or2 [x,2]-(cap_dy2-dis_deep2))
                    # battery cpacity
                    cap_dy2 =  dis_deep2
                    battery_stat2.append(cap_dy2)
            else: #battery can not work
                 # battery parameters
                en_dump2.append(0)   
                en_battery_grid2.append(0)
                en_battery_pv2.append(0)
                # power paremeters
                use_power_pv2.append(data_all_or2 [x,2])
                use_power_grid2.append(data_all_or2 [x,3]-data_all_or2 [x,2])
                # battery cpacity
                cap_dy2 =  cap_dy2
                battery_stat2.append(cap_dy2)
         
#the table of consumer2
#data_all_or2(time,hour,generation,consumption,surplus,grid consumption)
#use_power_pv2,use_power_grdi2
#battery_stat2,en_battery_grid2, en_battery_pv2, en_dump2,

#change the format of data
use_power_pv2=np.array(use_power_pv2)
use_power_grid2=np.array(use_power_grid2)
battery_stat2=np.array(battery_stat2)
en_battery_grid2=np.array(en_battery_grid2)
en_batteryp_v2=np.array(en_battery_pv2)
en_dump2=np.array(en_dump2)

head2=np.array(['time_day','hour','generation_pv','consumption','surplus','grid consumption','use_power_pv1','use_power_grid1',
               'battery_stat1','en_battery_grid1', 'en_battery_pv1', 'en_dump1'])
data2_power_battery=np.transpose(np.vstack((use_power_pv2,use_power_grid2,battery_stat2,en_battery_grid2,en_batteryp_v2,en_dump2))) 
data2=np.vstack((head2,np.hstack((data_all_or2,data2_power_battery))))


'FINAL RESULT FOR CUSTOMER 1 AND 2'
print(data1)
print(data2)

'DATA FILE1'
with open('data1.csv','w') as file1:
    writer = csv.writer(file1)
    writer.writerows(data1)

'DATA FILE2'
with open('data2.csv','w') as file2:
    writer = csv.writer(file2)
    writer.writerows(data2)

# Part of code of data process from master project

## data process 1

In [ ]:
"""Aim of this part of code:
    output pickle which contain the data of each meters"""

import pickle
import numpy as np
from datetime import datetime
import copy

load=open(r'C:\Users\xiaweijie\Desktop\唯有读书高\毕设\数据\smart_meter_data_new.pickle','rb')
o_data=pickle.load(load)


#put all data together
data=np.empty(shape=(0,5))
for i in range(len(o_data)):
    data=np.vstack((data,np.array(o_data[i])))

#Number of smart meters
n_meters=list(set(data[:,0]))

#organize the data
dict_meters={}
for y in n_meters:
    empty1=np.empty(shape=(0,5))
    for x in range(len(data)):
        if data[x,0]==y:
            empty1=np.vstack((empty1,data[x,:]))
            print(x,y)
    dict_meters[y]=empty1
    
file_dict_meters=open(r'C:\Users\xiaweijie\Desktop\唯有读书高\毕设\数据\file_dict.pickle','wb')
pickle.dump(dict_meters,file_dict_meters)
file_dict_meters.close()





## data process 2

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sat Nov 27 19:33:48 2021

@author: xiaweijie
"""

"""Aim of this part of code:
    process repeated value, lost value, outlier"""
import pickle
import numpy as np
import datetime
import pandas as pd
import copy

load2=open(r'C:\Users\xiaweijie\Desktop\唯有读书高\毕设\数据\file_dict.pickle','rb')
dict_meters=pickle.load(load2)


"Get the index of value lower than one could be discarded"
deleted_row={}
for x in range(len(dict_meters)):
    empty2=[]
    for y in range(len(dict_meters[x])):       
        if float(dict_meters[x][y,3])<10:
            empty2.append(y)
    deleted_row[x]=empty2


"delet empty data"
deleted_row[44]=[] #meter 44 has no empty value
for x in range(len(dict_meters)):
    dict_meters[x]=np.delete(dict_meters[x],(deleted_row[x]),axis = 0)

deleted_row_flu={}
for x in [77,58,44]:
    empty2=[]
    for y in range(len(dict_meters[x])):
        if float(dict_meters[x][y,3]) <= 6000 :
            empty2.append(y)
    deleted_row_flu[x]=empty2
    
for x in [77,58,44]:
    dict_meters[x]=np.delete(dict_meters[x],(deleted_row_flu[x]),axis = 0)           

'sorted by consumption'
'can not sort by consumption, duo to PV '
# for x in range(len(dict_meters)):
#     dict_meters[x]=np.array(pd.DataFrame(dict_meters[x]).sort_values(by=4,ascending=False))
    
# dict_meters2=dict_meters.copy()  
# 'calculate consumption for each gap'
# for x in range(len(dict_meters)): 
#     for y in range(len(dict_meters[x])):    
#         if y+1 < len(dict_meters[x]):
#             dict_meters[x][y+1,3]=float(dict_meters[x][y+1,3])-float(dict_meters[x][y,3])
#         else:
#             break

'Extract the part with time data'
'delete data without time'
deleted_row_nodata={}
for x in range(len(dict_meters)):
    empty3=[]
    for y in range(len(dict_meters[x])):  
        if len(dict_meters[x][y,2])<12:
            empty3.append(y)
    deleted_row_nodata[x]=empty3

# yaolao_data={}
# for x in range(len(dict_meters)):
#     yaolao_data[x]=dict_meters[x][deleted_row_nodata[x]]

# file_dict_laoshuju=open(r'C:\Users\xiaweijie\Desktop\唯有读书高\毕设\数据\file_dict_laoshuju.pickle','wb')
# pickle.dump(yaolao_data,file_dict_laoshuju)
# file_dict_laoshuju.close() 


'Extract the part with time data'
for x in range(len(dict_meters)):
    dict_meters[x]=np.delete(dict_meters[x],(deleted_row_nodata[x]),axis = 0)


'test kinds of date'
# data_len=[]
# for x in range(len(dict_meters)):
#     for y in range(len(dict_meters[x])):  
#         data_len.append(len(dict_meters[x][y,2]))
# types_data=list(set(data_len))
dict_meters_test=copy.deepcopy(dict_meters)

deleted_row_nodata2={}
'delet AM, PM data'
for x in range(len(dict_meters)):
    empty3_2=[]
    for y in range(len(dict_meters[x])): 
        if dict_meters[x][y,2][-2:] == 'AM':
            empty3_2.append(y)
        elif dict_meters[x][y,2][-2:] == 'PM':
            empty3_2.append(y)
    deleted_row_nodata2[x]=empty3_2  
    
for x in range(len(dict_meters)):
    dict_meters[x]=np.delete(dict_meters[x],(deleted_row_nodata2[x]),axis = 0)

"change to standard time"
time_structrue1="%d-%m-%Y %H:%M"
time_structrue2="%d-%m-%y %H:%M"
for x in range(len(dict_meters)):
    for y in range(len(dict_meters[x])): 
        try:
            dict_meters[x][y,2]=datetime.datetime.strptime(dict_meters[x][y,2],time_structrue1)
        except ValueError:
            dict_meters[x][y,2]=datetime.datetime.strptime(dict_meters[x][y,2],time_structrue2)


"Sort by date"
for x in range(len(dict_meters)):
    dict_meters[x]=np.array(pd.DataFrame(dict_meters[x]).sort_values(by=2,ascending=True))
    
file_dict_meters2=open(r'C:\Users\xiaweijie\Desktop\唯有读书高\毕设\数据\file_dict2.pickle','wb')
pickle.dump(dict_meters,file_dict_meters2)
file_dict_meters2.close() 


## data process 3

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sat Nov 27 23:39:42 2021

@author: xiaweijie
"""
import pickle
import numpy as np
from datetime import datetime
import copy
# import pandas as pd
# import string

load3=open(r'C:\Users\xiaweijie\Desktop\唯有读书高\毕设\数据\file_dict2.pickle','rb')
dict_meters=pickle.load(load3)


'delet repeated value'
deleted_row_repeated={}
for x in range(len(dict_meters)):
    empty4=[]
    for y in range(len(dict_meters[x])): 
        if y+1 < len(dict_meters[x]):
            if dict_meters[x][y,2]==dict_meters[x][y+1,2]:
                empty4.append(y)
    deleted_row_repeated[x]=empty4

for x in range(len(dict_meters)):
    dict_meters[x]=np.delete(dict_meters[x],(deleted_row_repeated[x]),axis = 0)


'add index to all data'
time_lost=np.empty(shape=(0,3))
for x in range(len(dict_meters)):
    for y in range(len(dict_meters[x])): 
        if y+1 < len(dict_meters[x]):
            t_gap=str(dict_meters[x][y+1,2]-dict_meters[x][y,2])
            if t_gap=='0 days 00:15:00':
                pass
            else:
                time_lost=np.vstack((time_lost,[x,y,(float(t_gap[0])*24*60+60*float(t_gap[-8]+t_gap[-7])+float(t_gap[-5]+t_gap[-4]))/15]))
        
"""
following task:
Aim: creating a virtual weekful data for each smart meter
"""
'abstract data time'
for x in range(len(dict_meters)):
    for y in range(len(dict_meters[x])):
        date_day=str(dict_meters[x][y,2])
        dict_meters[x][y,4]=date_day[0:10]
        dict_meters[x][y,4] = 1 + datetime.strptime(dict_meters[x][y,4],"%Y-%m-%d").weekday()

'calculate power consumption'
# for x in range(len(dict_meters)):
#     for y in range(len(dict_meters[x])):
#         for z in range(len(time_lost)):
#             if x==time_lost[z,0] and y== time_lost[z+1,1]:
#                 pass
#             else:
#                 if y-1<0:
#                     pass
#                 else:
#                     dict_meters[x][y,3]=str(float(dict_meters[x][y,3])-float(dict_meters[x][y-1,3]))

dict_meters_mid=copy.deepcopy(dict_meters)

for x in range(len(dict_meters)):
    for y in range(len(dict_meters[x])):
          if y-1<0:
              pass
          else:
              dict_meters[x][y,3]=str(float(dict_meters_mid[x][y,3])-float(dict_meters_mid[x][y-1,3]))
              
'delete useless(too long time gap) value'
time_lost_dict={}
for y in range(99):
    mid_array_time=[]
    for x in range(len(time_lost)):
        if y==time_lost[x,0]:
            mid_array_time.append(int(time_lost[x,1]+1))
            mid_array_time.append(0)
        time_lost_dict[y]=mid_array_time

for x in range(len(dict_meters)):
    dict_meters[x]=np.delete(dict_meters[x],(time_lost_dict[x]),axis = 0)
    
file_dict_meters3=open(r'C:\Users\xiaweijie\Desktop\唯有读书高\毕设\数据\file_dict3.pickle','wb')
pickle.dump(dict_meters,file_dict_meters3)
file_dict_meters3.close() 



## data process 4

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Dec  1 20:08:58 2021

@author: xiaweijie
"""

import pickle
import numpy as np
from datetime import datetime

load4=open(r'C:\Users\xiaweijie\Desktop\唯有读书高\毕设\数据\file_dict3.pickle','rb')
dict_meters=pickle.load(load4)

'split time of a day out'
for x in range(len(dict_meters)):
    for y in range(len(dict_meters[x])):
        date_time=str(dict_meters[x][y,2])
        dict_meters[x][y,1]=date_time[11:20]

'allocate data to each day of the week'
dict_meters_day={}
for x in range(len(dict_meters)):
    dict_day={}
    for y in [1,2,3,4,5,6,7]:
        data_of_day=np.empty(shape=(0,5))
        for z in range(len(dict_meters[x])):
            if dict_meters[x][z,4] == y:
                data_of_day=np.vstack((data_of_day,dict_meters[x][z]))
        dict_day[y]=data_of_day
    dict_meters_day[x]=dict_day
    
file_dict_meters4=open(r'C:\Users\xiaweijie\Desktop\唯有读书高\毕设\数据\file_dict4.pickle','wb')
pickle.dump(dict_meters_day,file_dict_meters4)
file_dict_meters4.close() 


## data analysis 1

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Dec  1 20:30:24 2021

@author: xiaweijie
"""

import pickle
import numpy as np
import pandas as pd
from datetime import datetime


load5=open(r'C:\Users\xiaweijie\Desktop\唯有读书高\毕设\数据\file_dict4.pickle','rb')
dict_meters_day=pickle.load(load5)

'average the data of a day'
dict_meters_day_ave={}
n_gaps=np.array(list(set(dict_meters_day[0][1][:,1])))
time_structrue_a1="%H:%M:%S"

for x in range(len(dict_meters_day)):
    dict_meter_day_gap={}
    for y in range(len(dict_meters_day[x])):
        data_of_each_day=np.empty(shape=(0,4))
        for z in range(len(n_gaps)):
            midle_valule_gap=[]
            for a in range(len(dict_meters_day[x][y+1])):
                if n_gaps[z]==dict_meters_day[x][y+1][a,1]: #这里要确认下
                    midle_valule_gap.append(float(dict_meters_day[x][y+1][a,3]))
            time_gap_ave=np.sum(midle_valule_gap)/len(midle_valule_gap)
            array_day_each_gap=np.array([x,n_gaps[z],
                                         dict_meters_day[x][y+1][a,4],
                                          time_gap_ave])
            data_of_each_day=np.vstack((data_of_each_day,array_day_each_gap))
        dict_meter_day_gap[y+1]=pd.DataFrame(data_of_each_day).sort_values(by=1)
    dict_meters_day_ave[x]=dict_meter_day_gap
    
    
data_day_week=open(r'C:\Users\xiaweijie\Desktop\唯有读书高\毕设\数据\data\data_day_week.pickle','wb')
pickle.dump(dict_meters_day_ave,data_day_week)
data_day_week.close() 


## data analysis 2

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Dec  1 21:46:34 2021

@author: xiaweijie
"""

import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from datetime import datetime


load_data=open(r'C:\Users\xiaweijie\Desktop\唯有读书高\毕设\数据\data\data_day_week.pickle','rb')
dict_meters_day=pickle.load(load_data)

# data_11=[]
# for i in range(len(dict_meters_day[0][1])):
    # if 2*i >len(dict_meters_day[0][1]):
    #     pass
    # else:
    # a=1*i
    # if a >= len(dict_meters_day[0][1]):
        # pass
    # else:
        # data_11.append(float(dict_meters_day[0][7].loc[a,3]))

# plt.plot(data_11)
# for x in range(len(dict_meters_day)):
#     for z in range(96):
#         for y in range(len(dict_meters_day[x])): #y+1

#             if 
dict_meters_day_weekday={}
dict_meters_day_weekend={}
def str_to_float_array(x):
    for i in range(len(x)):
        x[i]=float(x[i])
    return x

def ave_weekday(x):
    for i in range(len(x)):
        x[i]=x[i]/5
    return x

def ave_weekend(x):
    for i in range(len(x)):
        x[i]=x[i]/2
    return x

#range(len(dict_meters_day))
for x in range(len(dict_meters_day)):
    mid_var=0
    for y in [1,2,3,4,5]:
        mid_var = mid_var + str_to_float_array(dict_meters_day[x][y].loc[:,3])
        print(x,y)
    # print(ave_weekday(mid_var))
    dict_meters_day_weekday[x]=np.transpose(np.vstack((dict_meters_day[0][1].loc[:,1],ave_weekday(mid_var))))
    
#range(len(dict_meters_day))
for x in range(len(dict_meters_day)):
    mid_var=0
    for y in [6,7]:
        mid_var = mid_var + str_to_float_array(dict_meters_day[x][y].loc[:,3])
        print(x,y)
    dict_meters_day_weekend[x]=np.transpose(np.vstack((dict_meters_day[0][1].loc[:,1],ave_weekend(mid_var))))

'16 周五 晚上9点15 特别巨大的数据，为什么会这样？'
dict_meters_day_weekday[16][85,1]=(dict_meters_day_weekday[16][84,1]+dict_meters_day_weekday[16][86,1])/2
dict_meters_day_weekday[16][1,1]=(dict_meters_day_weekday[16][0,1]+dict_meters_day_weekday[16][2,1])/2
dict_meters_day_weekday[16][66,1]=(dict_meters_day_weekday[16][65,1]+dict_meters_day_weekday[16][67,1])/2
dict_meters_day_weekend[16][13,1]=(dict_meters_day_weekend[16][12,1]+dict_meters_day_weekend[16][14,1])/2

dict_meters_day_weekend[34][36,1]=0.25
dict_meters_day_weekend[34][35,1]=(dict_meters_day_weekend[34][36,1]+dict_meters_day_weekend[34][36,1])/2
dict_meters_day_weekend[34][37,1]=(dict_meters_day_weekend[34][36,1]+dict_meters_day_weekend[34][38,1])/2

#range(len(dict_meters_day_weekend))
for x in [34]:
    string='power for weekday'+' '+str(x)
    plt.title(string) 
    plt.xlabel("x time") 
    plt.ylabel("y power")
    plt.plot(dict_meters_day_weekday[x][:,1])
    plt.show()
    string='power for weekend'+' '+str(x)
    plt.title(string) 
    plt.xlabel("x time") 
    plt.ylabel("y power")
    plt.plot(dict_meters_day_weekend[x][:,1])
    plt.show()
    print(x)
    
# print('finding number')
# for x in dict_meters_day[44]:
#     for y in range(len(dict_meters_day[44][x])):
#         if dict_meters_day[44][x].loc[y,3] == max(dict_meters_day[44][x].loc[:,3]):
#             print(x,y,dict_meters_day[44][x].loc[y,3],dict_meters_day[44][x].loc[y,2])
            
        